##### Solve the IBM Quantum Challenge Fall 2021 problem and tackle more advanced versions.

# Challenge 4b: Battery revenue optimization with Qiskit knapsack class #

<div class="alert alert-block alert-success">

**Challenge 4b** <br>
We will optimize the battery schedule using Qiskit optimization knapsack class with QAOA to maximize the total return with a cost within $C_{max}$ under the following conditions;
    <br>
- the time window $t = 7$<br>
- the daily return $\lambda_{1} = [5, 3, 3, 6, 9, 7, 1]$<br>
- the daily return $\lambda_{2} = [8, 4, 5, 12, 10, 11, 2]$<br>
- the daily degradation for the battery $c_{1} = [1, 1, 2, 1, 1, 1, 2]$<br>
- the daily degradation for the battery $c_{2} = [3, 2, 3, 2, 4, 3, 3]$<br>
- $C_{max} = 16$<br>
<br>
    
Your task is to find the argument, `values`, `weights`, and `max_weight` used for the Qiskit optimization knapsack class, to get a solution which "0" denote the choice of market $M_{1}$, and "1" denote the choice of market $M_{2}$. We will check your answer with another data set of $\lambda_{1}, \lambda_{2}, c_{1}, c_{2}, C_{max}$.
    <br>
You can receive a badge for solving all the challenge exercises up to 4b.

</div>

In [115]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA
from qiskit.primitives import Sampler
from qiskit_algorithms.optimizers import COBYLA
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.applications import Knapsack
import numpy as np

In [ ]:
L1 = [5,3,3,6,9,7,1]
L2 = [8,4,5,12,10,11,2]
C1 = [1,1,2,1,1,1,2]
C2 = [3,2,3,2,4,3,3]
C_max = 16

In [ ]:
def knapsack_argument(L1, L2, C1, C2, C_max):
      
    ##############################
    # Provide your code here
    values = list(map(lambda x, y: x - y, L2, L1))
    weights = list(map(lambda x, y: x - y, C2, C1))
    max_weight = C_max - sum(C1)
    #
    ##############################
    return values, weights, max_weight
    
values, weights, max_weight = knapsack_argument(L1, L2, C1, C2, C_max)
print(values, weights, max_weight)
prob = Knapsack(values = values, weights = weights, max_weight = max_weight)
qp = prob.to_quadratic_program()
qp

[3, 1, 2, 6, 1, 4, 1] [2, 1, 1, 1, 3, 2, 1] 7


<QuadraticProgram: maximize 3*x_0 + x_1 + 2*x_2 + 6*x_3 + x_4 + 4*x_5 + x_6, 7 variables, 1 constraints, 'Knapsack'>

In [ ]:
# QAOA

mes = QAOA(sampler=Sampler(), optimizer=COBYLA())
meo = MinimumEigenOptimizer(min_eigen_solver=mes)

result = meo.solve(qp)
print('result:', result.x)

item = np.array(result.x)
revenue=0
for i in range(len(item)):
    if item[i]==0:
        revenue+=L1[i]
    else:
        revenue+=L2[i]

print('total revenue:', revenue)

C:\Users\forca\AppData\Local\Temp\ipykernel_18416\341595638.py:3: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  mes = QAOA(sampler=Sampler(), optimizer=COBYLA())


result: [1. 1. 1. 1. 0. 1. 0.]
total revenue: 50


---

##### Evolve from the basic 2-state knapsack to the multi-state knapsack.

In [ ]:
L1 = [3, 5, 2, 4]
L2 = [7, 1, 6, 8]
L3 = [8, 2, 4, 5]
C1 = [2, 3, 5, 4]
C2 = [4, 2, 1, 5]
C3 = [3, 4, 2, 6]

C_max = 16

In [68]:
def solve_multi_state_knapsack(profit_values, cost_values, max_total_cost):
    """
    Solve a multi-state knapsack problem using a classical backtracking algorithm.
    
    Args:
        profit_values (2d list of float): Profit values for the possible states.
        cost_values (2d list of float): Cost values for the possible states.
        max_total_cost (float): Maximum allowable total cost.
        
    Returns:
        list: The optimal selection of states for each item.
        float: The maximum total profit.

        
    # len(profit_values) == len(cost_values)
    # len(profit_values[0]) == len(cost_values[0])

    """
    n = len(profit_values[0])  # Number of items
    num_states = len(profit_values)
    
    def backtrack(index, current_cost, current_profit, current_solution):
        """
        Recursive backtracking function to search for the optimal solution.
        
        Args:
            index (int): Current item index being processed.
            current_cost (float): Current total cost of the solution path.
            current_profit (float): Current total profit of the solution path.
            current_solution (list of int): Current list of selected states for items.
        
        Returns:
            list: The best solution (state selection) for the current path.
            float: The total profit of the best solution.
        """
        if index == n:  # Base case: all items are processed
            return current_solution, current_profit
        
        best_solution, best_profit = None, -float('inf')
        
        for state in range(num_states):  # possible states for each item
            new_cost = current_cost + cost_values[state][index]
            new_profit = current_profit + profit_values[state][index]
            
            if new_cost <= max_total_cost:  # Prune paths exceeding the cost limit
                solution, profit = backtrack(
                    index + 1, 
                    new_cost, 
                    new_profit, 
                    current_solution + [state]
                )
                
                if solution and profit > best_profit:
                    best_solution, best_profit = solution, profit
        
        return best_solution, best_profit

    # Start the backtracking process from index 0
    solution, max_profit = backtrack(0, 0, 0, [])
    return solution, max_profit

In [3]:
def formulate_qp_multi_state_knapsack(profit_values, cost_values, max_total_cost):
    """
    Formulate the multi-state knapsack problem as a quadratic program using Qiskit's QuadraticProgram.
    
    Args:
        profit_values (2d list of float): Profit values for the possible states.
        cost_values (2d list of float): Cost values for the possible states.
        max_total_cost (float): Maximum allowable total cost.
    
    Returns:
        QuadraticProgram: The formulated quadratic program for QAOA-based solution.

        
    # len(profit_values) == len(cost_values)
    # len(profit_values[0]) == len(cost_values[0])

    """
    
    n = len(profit_values[0])  # Number of items
    num_states = len(profit_values)  # Number of possible states per item

    qp = QuadraticProgram("Multi-State Knapsack Problem")

    # Define binary variables for each item's possible state (states per item)
    for i in range(n):
        for s in range(num_states):
            qp.binary_var(name=f"state_{s}_{i}") 

    # Objective function: maximize profits
    linear_terms = {}
    for i in range(n):
        for s in range(num_states):
            linear_terms[f"state_{s}_{i}"] = profit_values[s][i] 

    qp.maximize(linear=linear_terms)

    # Constraint 1: Each item must be assigned to exactly one state
    for i in range(n):
        qp.linear_constraint(
            linear={f"state_{s}_{i}": 1 for s in range(num_states)},
            sense="==",
            rhs=1,
            name=f"state_assignment_constraint_{i}",
        )

    # Constraint 2: Total cost must not exceed the maximum allowable cost
    cost_terms = {}
    for i in range(n):
        for s in range(num_states):
            cost_terms[f"state_{s}_{i}"] = cost_values[s][i]

    qp.linear_constraint(
        linear=cost_terms,
        sense="<=",
        rhs=max_total_cost,
        name="total_cost_constraint",
    )

    return qp



In [4]:
def solve_qp_with_qiskit(profit_values, cost_values, max_total_cost):
    """
    Solve the quadratic program using Qiskit's optimization solvers.

    Returns:
        dict: Solution details including selected items and total revenue.
    """
    qp = formulate_qp_multi_state_knapsack(profit_values, cost_values, max_total_cost)

    print(qp.prettyprint())
    
    qaoa = QAOA(sampler=Sampler(), optimizer=COBYLA())
    
    optimizer = MinimumEigenOptimizer(qaoa)

    result = optimizer.solve(qp)
    
    solution = result.x
    
    # Extract results
    total_profit = 0
    assignments = []

    
    n = len(profit_values[0])  # Number of items
    num_states = len(profit_values)  # Number of possible states per item


    for i in range(n):
        for s in range(num_states):
            if solution[num_states * i + s] == 1:
                total_profit += profit_values[s][i]
                assignments.append(s)


    return {
        "assignments": assignments,
        "total_profit": total_profit,
        "ansatz": qaoa.ansatz
    }

In [ ]:
profits = [[3, 5, 2, 4],
           [7, 1, 6, 8],
           [8, 2, 4, 5]]
costs = [[2, 3, 5, 4],
         [4, 2, 1, 5],
         [3, 4, 2, 6]]


max_total_cost = 10


# Classical solution
solution, max_profit = solve_multi_state_knapsack(profits, costs, max_total_cost)
print(f"Optimal state selection: {solution}, Maximum profit: {max_profit}")

# Quantum solution
result = solve_qp_with_qiskit(profits, costs, max_total_cost)
print(f"Optimal state selection: {result["assignments"]}, Maximum profit: {result["total_profit"]}")

Optimal state selection: [2, 1, 1, 0], Maximum profit: 19
Problem name: Multi-State Knapsack Problem

Maximize
  3*state_0_0 + 5*state_0_1 + 2*state_0_2 + 4*state_0_3 + 7*state_1_0
  + state_1_1 + 6*state_1_2 + 8*state_1_3 + 8*state_2_0 + 2*state_2_1
  + 4*state_2_2 + 5*state_2_3

Subject to
  Linear constraints (5)
    state_0_0 + state_1_0 + state_2_0 == 1  'state_assignment_constraint_0'
    state_0_1 + state_1_1 + state_2_1 == 1  'state_assignment_constraint_1'
    state_0_2 + state_1_2 + state_2_2 == 1  'state_assignment_constraint_2'
    state_0_3 + state_1_3 + state_2_3 == 1  'state_assignment_constraint_3'
    2*state_0_0 + 3*state_0_1 + 5*state_0_2 + 4*state_0_3 + 4*state_1_0
    + 2*state_1_1 + state_1_2 + 5*state_1_3 + 3*state_2_0 + 4*state_2_1
    + 2*state_2_2 + 6*state_2_3 <= 10  'total_cost_constraint'

  Binary variables (12)
    state_0_0 state_1_0 state_2_0 state_0_1 state_1_1 state_2_1 state_0_2
    state_1_2 state_2_2 state_0_3 state_1_3 state_2_3



C:\Users\forca\AppData\Local\Temp\ipykernel_23196\657011922.py:12: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  qaoa = QAOA(sampler=Sampler(), optimizer=COBYLA())


Optimal state selection: [2, 1, 1, 0], Maximum profit: 19


---
##### Evolve from the multi-state knapsack to the multi-state multi-knapsack.

In [69]:
def solve_multi_state_multi_knapsack(profit_values, cost_values, max_total_costs):
    """
    Solve a multi-state multi-knapsack problem using a classical backtracking algorithm.
    
    Args:
        profit_values (2D list of float): Profit values for the possible states for each item.
        cost_values (3D list of float): Cost values for the possible states for each item in each knapsack.
                                        cost_values[s][i][k] means the cost of item i in state s for knapsack k.
        max_total_costs (list of float): Maximum allowable total cost for each knapsack.
        
    Returns:
        list: The optimal selection of states for each item.
        list: The allocation of each item to knapsacks.
        float: The maximum total profit.
        
    # len(profit_values) == len(cost_values)
    # len(profit_values[0]) == len(cost_values[0])
    # len(max_total_costs) == number of knapsacks
    """
    n = len(profit_values[0])  # Number of items
    num_knapsacks = len(max_total_costs)
    num_states = len(profit_values)
    
    def backtrack(index, current_costs, current_profit, current_solution, current_knapsack_allocation):
        """
        Recursive backtracking function to search for the optimal solution.
        
        Args:
            index (int): Current item index being processed.
            current_costs (list of float): Current total cost used in each knapsack.
            current_profit (float): Current total profit of the solution path.
            current_solution (list of int): Current list of selected states for items.
            current_knapsack_allocation (list of int): Current list of knapsack assignments for items.
        
        Returns:
            list: The best solution (state selection) for the current path.
            list: The best knapsack allocation for the current path.
            float: The total profit of the best solution.
        """
        if index == n:  # Base case: all items are processed
            return current_solution, current_knapsack_allocation, current_profit
        
        best_solution, best_allocation, best_profit = None, None, -float('inf')
        
        for state in range(num_states):  # Possible states for each item
            for knapsack in range(num_knapsacks):  # Possible knapsacks for each item
                new_costs = current_costs[:]
                new_costs[knapsack] += cost_values[state][index][knapsack]
                
                if all(new_costs[k] <= max_total_costs[k] for k in range(num_knapsacks)):  # Prune paths exceeding the cost limits
                    solution, allocation, profit = backtrack(
                        index + 1, 
                        new_costs, 
                        current_profit + profit_values[state][index], 
                        current_solution + [state], 
                        current_knapsack_allocation + [knapsack]
                    )
                    
                    if profit > best_profit:
                        best_solution, best_allocation, best_profit = solution, allocation, profit
        
        return best_solution, best_allocation, best_profit

    # Start the backtracking process from index 0
    initial_costs = [0] * num_knapsacks
    solution, knapsack_allocation, max_profit = backtrack(0, initial_costs, 0, [], [])
    return solution, knapsack_allocation, max_profit



In [70]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA
from qiskit.primitives import Sampler
from qiskit_algorithms.optimizers import COBYLA
from qiskit_optimization import QuadraticProgram

def formulate_qp_multi_state_multi_knapsack(profit_values, cost_values, max_total_costs):
    """
    Formulate the multi-state multi-knapsack problem as a quadratic program using Qiskit's QuadraticProgram.
    
    Args:
        profit_values (2D list of float): Profit values for the possible states for each item.
        cost_values (3D list of float): Cost values for the possible states for each item in each knapsack.
                                        cost_values[s][i][k] means the cost of item i in state s for knapsack k.
        max_total_costs (list of float): Maximum allowable total cost for each knapsack.
    
    Returns:
        QuadraticProgram: The formulated quadratic program for QAOA-based solution.
    """
    n = len(profit_values[0])  # Number of items
    num_states = len(profit_values)  # Number of possible states per item
    num_knapsacks = len(max_total_costs)  # Number of knapsacks
    
    qp = QuadraticProgram("Multi-State Multi-Knapsack Problem")

    # Step 1: Define binary variables for each item's possible state and knapsack allocation
    for i in range(n):
        for s in range(num_states):
            for k in range(num_knapsacks):
                qp.binary_var(name=f"x_{s}_{i}_{k}") 

    # Step 2: Objective function: maximize profits
    linear_terms = {}
    for i in range(n):
        for s in range(num_states):
            for k in range(num_knapsacks):
                linear_terms[f"x_{s}_{i}_{k}"] = profit_values[s][i]  # Profit of item i in state s

    qp.maximize(linear=linear_terms)

    # Step 3: Constraint 1: Each item must be assigned to exactly one state in one knapsack
    for i in range(n):
        qp.linear_constraint(
            linear={f"x_{s}_{i}_{k}": 1 for s in range(num_states) for k in range(num_knapsacks)},
            sense="==",
            rhs=1,
            name=f"state_and_knapsack_assignment_constraint_{i}",
        )

    # Step 4: Constraint 2: Total cost for each knapsack must not exceed the maximum allowable cost
    for k in range(num_knapsacks):
        cost_terms = {}
        for i in range(n):
            for s in range(num_states):
                cost_terms[f"x_{s}_{i}_{k}"] = cost_values[s][i][k]  # Cost of item i in state s for knapsack k
        qp.linear_constraint(
            linear=cost_terms,
            sense="<=",
            rhs=max_total_costs[k],
            name=f"total_cost_constraint_knapsack_{k}",
        )

    return qp


def solve_qp_with_qiskit(profit_values, cost_values, max_total_costs):
    """
    Solve the quadratic program using Qiskit's optimization solvers.

    Args:
        profit_values (2D list of float): Profit values for each item in each state.
        cost_values (3D list of float): Cost values for each item in each state for each knapsack.
        max_total_costs (list of float): Maximum allowable total cost for each knapsack.

    Returns:
        dict: Solution details including selected items and total revenue.
    """
    qp = formulate_qp_multi_state_multi_knapsack(profit_values, cost_values, max_total_costs)

    print(qp.prettyprint())
    
    qaoa = QAOA(sampler=Sampler(), optimizer=COBYLA())
    optimizer = MinimumEigenOptimizer(qaoa)
    result = optimizer.solve(qp)
    solution = result.x

    total_profit = 0
    assignments = []

    
    n = len(profit_values[0])  # Number of items
    num_knapsacks = len(max_total_costs)
    num_states = len(profit_values)

    for i in range(n):  # For each item
        for s in range(num_states):  # For each state
            for k in range(num_knapsacks):  # For each knapsack
                index = (num_knapsacks * num_states * i) + (num_knapsacks * s) + k
                if solution[index] == 1:
                    total_profit += profit_values[s][i]
                    assignments.append((i, s, k))  # (item, state, knapsack)

    return {
        "assignments": assignments,
        "total_profit": total_profit,
        "ansatz": qaoa.ansatz
    }



In [ ]:
profit_values = [
    [5, 10],   # Profits if item i is in state 0
    [8, 15],   # Profits if item i is in state 1
    [12, 20]   # Profits if item i is in state 2
]

cost_values = [
    [  # Costs for items if in state 0
        [10, 12], [15, 18], 
    ],
    [  # Costs for items if in state 1
        [11, 13], [16, 19], 
    ],
    [  # Costs for items if in state 2
        [14, 17], [20, 22], 
    ]
]

max_total_costs = [20, 25]  # Max cost for knapsack 0 and 1

# Solve the problem
solution, knapsack_allocation, max_profit = solve_multi_state_multi_knapsack(profit_values, cost_values, max_total_costs)

print(f"Optimal state selection: {solution}")
print(f"Knapsack allocation: {knapsack_allocation}")
print(f"Maximum profit: {max_profit}")

result = solve_qp_with_qiskit(profit_values, cost_values, max_total_costs)
print("Assignments (item, state, knapsack):", result['assignments'])
print("Total profit:", result['total_profit'])


---
##### Learn the implementation of Qiskit QAOA, QuadraticProgramToQubo, and QUBO.

In [89]:
from qiskit_optimization.converters import QuadraticProgramToQubo


profits = [[3, 5, 2, 4],
           [7, 1, 6, 8],
           [8, 2, 4, 5]]
costs = [[2, 3, 5, 4],
         [4, 2, 1, 5],
         [3, 4, 2, 6]]


max_total_cost = 10


original_problem = formulate_qp_multi_state_knapsack(profits, costs, max_total_cost)
print(original_problem.prettyprint())

Problem name: Multi-State Knapsack Problem

Maximize
  3*state_0_0 + 5*state_0_1 + 2*state_0_2 + 4*state_0_3 + 7*state_1_0
  + state_1_1 + 6*state_1_2 + 8*state_1_3 + 8*state_2_0 + 2*state_2_1
  + 4*state_2_2 + 5*state_2_3

Subject to
  Linear constraints (5)
    state_0_0 + state_1_0 + state_2_0 == 1  'state_assignment_constraint_0'
    state_0_1 + state_1_1 + state_2_1 == 1  'state_assignment_constraint_1'
    state_0_2 + state_1_2 + state_2_2 == 1  'state_assignment_constraint_2'
    state_0_3 + state_1_3 + state_2_3 == 1  'state_assignment_constraint_3'
    2*state_0_0 + 3*state_0_1 + 5*state_0_2 + 4*state_0_3 + 4*state_1_0
    + 2*state_1_1 + state_1_2 + 5*state_1_3 + 3*state_2_0 + 4*state_2_1
    + 2*state_2_2 + 6*state_2_3 <= 10  'total_cost_constraint'

  Binary variables (12)
    state_0_0 state_1_0 state_2_0 state_0_1 state_1_1 state_2_1 state_0_2
    state_1_2 state_2_2 state_0_3 state_1_3 state_2_3



In [90]:
# define a problem
conv = QuadraticProgramToQubo()
qubo = conv.convert(original_problem)
print(qubo.prettyprint())

Problem name: Multi-State Knapsack Problem

Minimize
  280*state_0_0^2 + 672*state_0_0*state_0_1 + 1120*state_0_0*state_0_2
  + 896*state_0_0*state_0_3 + 1008*state_0_0*state_1_0 + 448*state_0_0*state_1_1
  + 224*state_0_0*state_1_2 + 1120*state_0_0*state_1_3 + 784*state_0_0*state_2_0
  + 896*state_0_0*state_2_1 + 448*state_0_0*state_2_2 + 1344*state_0_0*state_2_3
  + 224*state_0_0*total_cost_constraint@int_slack@0
  + 448*state_0_0*total_cost_constraint@int_slack@1
  + 896*state_0_0*total_cost_constraint@int_slack@2
  + 672*state_0_0*total_cost_constraint@int_slack@3 + 560*state_0_1^2
  + 1680*state_0_1*state_0_2 + 1344*state_0_1*state_0_3
  + 784*state_0_1*state_1_1 + 336*state_0_1*state_1_2 + 1680*state_0_1*state_1_3
  + 1456*state_0_1*state_2_1 + 672*state_0_1*state_2_2
  + 2016*state_0_1*state_2_3 + 336*state_0_1*total_cost_constraint@int_slack@0
  + 672*state_0_1*total_cost_constraint@int_slack@1
  + 1344*state_0_1*total_cost_constraint@int_slack@2
  + 1008*state_0_1*total_cost_c

##### QuadraticProgramToQubo

In [91]:
profits = [[3, 5, 2],
           [7, 1, 6],
           [8, 2, 4]]
costs = [[2, 3, 5],
         [4, 2, 1],
         [3, 4, 2]]


max_total_cost = 10


original_problem = formulate_qp_multi_state_knapsack(profits, costs, max_total_cost)
print(original_problem.prettyprint())

Problem name: Multi-State Knapsack Problem

Maximize
  3*state_0_0 + 5*state_0_1 + 2*state_0_2 + 7*state_1_0 + state_1_1
  + 6*state_1_2 + 8*state_2_0 + 2*state_2_1 + 4*state_2_2

Subject to
  Linear constraints (4)
    state_0_0 + state_1_0 + state_2_0 == 1  'state_assignment_constraint_0'
    state_0_1 + state_1_1 + state_2_1 == 1  'state_assignment_constraint_1'
    state_0_2 + state_1_2 + state_2_2 == 1  'state_assignment_constraint_2'
    2*state_0_0 + 3*state_0_1 + 5*state_0_2 + 4*state_1_0 + 2*state_1_1
    + state_1_2 + 3*state_2_0 + 4*state_2_1 + 2*state_2_2
    <= 10  'total_cost_constraint'

  Binary variables (9)
    state_0_0 state_1_0 state_2_0 state_0_1 state_1_1 state_2_1 state_0_2
    state_1_2 state_2_2



Convert linear inequality constraints to penalty terms of the objective function.

There are some linear constraints which do not require slack variables to
construct penalty terms [1]. This class supports the following inequality constraints.

$$
\begin{array}{}
\text { Inequality constraint } & & \text { Penalty term } \\
x \leq y & \rightarrow  & P(x-x y) \\
x \geq y & \rightarrow  & P(y-x y) \\
\sum_{i=1}^n x_i \leq 1, n \geq 2 & \rightarrow & P \sum_{i, j : i < j} x_i x_j\\
\sum_{i=1}^n x_i \geq n-1, n \geq 2 & \rightarrow & P \sum_{i, j : i < j} (1 - x_i) (1 - x_j)
\end{array}
$$

Note that x, y, z and $x_i$ are binary variables, and P is a penalty factor,
where the value of P is automatically determined or supplied by users.

If constraints match with any of the patterns, they are converted into penalty terms and added
to the objective function. Otherwise, constraints are kept as is.

References:
    [1]: Fred Glover, et al. (2019),
            A Tutorial on Formulating and Using QUBO Models,
            arXiv:1811.11538 <https://arxiv.org/abs/1811.11538>

In [92]:
from qiskit_optimization.converters.linear_inequality_to_penalty import LinearInequalityToPenalty

binary_penalty_problem = LinearInequalityToPenalty().convert(original_problem)
print(binary_penalty_problem.prettyprint())

Problem name: Multi-State Knapsack Problem

Maximize
  3*state_0_0 + 5*state_0_1 + 2*state_0_2 + 7*state_1_0 + state_1_1
  + 6*state_1_2 + 8*state_2_0 + 2*state_2_1 + 4*state_2_2

Subject to
  Linear constraints (4)
    state_0_0 + state_1_0 + state_2_0 == 1  'state_assignment_constraint_0'
    state_0_1 + state_1_1 + state_2_1 == 1  'state_assignment_constraint_1'
    state_0_2 + state_1_2 + state_2_2 == 1  'state_assignment_constraint_2'
    2*state_0_0 + 3*state_0_1 + 5*state_0_2 + 4*state_1_0 + 2*state_1_1
    + state_1_2 + 3*state_2_0 + 4*state_2_1 + 2*state_2_2
    <= 10  'total_cost_constraint'

  Binary variables (9)
    state_0_0 state_1_0 state_2_0 state_0_1 state_1_1 state_2_1 state_0_2
    state_1_2 state_2_2



In [93]:
from qiskit_optimization.converters.inequality_to_equality import InequalityToEquality

equality_problem = InequalityToEquality(mode="integer").convert(binary_penalty_problem)
print(equality_problem.prettyprint())

Problem name: Multi-State Knapsack Problem

Maximize
  3*state_0_0 + 5*state_0_1 + 2*state_0_2 + 7*state_1_0 + state_1_1
  + 6*state_1_2 + 8*state_2_0 + 2*state_2_1 + 4*state_2_2

Subject to
  Linear constraints (4)
    state_0_0 + state_1_0 + state_2_0 == 1  'state_assignment_constraint_0'
    state_0_1 + state_1_1 + state_2_1 == 1  'state_assignment_constraint_1'
    state_0_2 + state_1_2 + state_2_2 == 1  'state_assignment_constraint_2'
    2*state_0_0 + 3*state_0_1 + 5*state_0_2 + 4*state_1_0 + 2*state_1_1
    + state_1_2 + 3*state_2_0 + 4*state_2_1 + 2*state_2_2
    + total_cost_constraint@int_slack == 10  'total_cost_constraint'

  Integer variables (1)
    0 <= total_cost_constraint@int_slack <= 10

  Binary variables (9)
    state_0_0 state_1_0 state_2_0 state_0_1 state_1_1 state_2_1 state_0_2
    state_1_2 state_2_2



### 🔍 **1. `_convert_var`**  
**Purpose:**  
Integer variable $ x \in [l, u] $ is converted to binary variables.  

**Logic:**  
1. **Range calculation**:  
   $$
   \text{var\_range} = u - l
   $$  
2. **Binary variable count**:  
   $$
   \text{power} = \lfloor \log_2(\text{var\_range}) \rfloor
   $$  
3. **Leftover coefficient**:  
   $$
   \text{bounded\_coef} = \text{var\_range} - (2^{\text{power}} - 1)
   $$  
4. **Binary variables and coefficients**:  
   $$
   \{(x@0, 2^0), (x@1, 2^1), \cdots, (x@(\text{power} - 1), 2^{\text{power}-1}), (x@\text{power}, \text{bounded\_coef})\}
   $$  

**Example:**  
If $ x \in [0, 10] $, then:  
$$
\{(x@0, 1), (x@1, 2), (x@2, 4), (x@3, 3)\}
$$  

### 🔍 **2. `_convert_linear_coefficients_dict`**  
**Purpose:**  
Convert linear coefficients for integer variables.  

**Logic:**  
1. For each original variable $ x $ with coefficient $ v $, apply to binary variables:  
   $$
   \text{new coefficient for } y = v \cdot \text{coefficient of } y
   $$  
2. **Shift constant** if variable $ x $ has a lower bound $ l $:  
   $$
   \text{constant} += v \cdot l
   $$  

**Example:**  
For $ v = 5 $ and $ l = 2 $, shift constant:  
$$
\text{constant} += 5 \cdot 2 = 10
$$

In [94]:
from qiskit_optimization.converters.integer_to_binary import IntegerToBinary

binary_problem = IntegerToBinary().convert(equality_problem)
print(binary_problem.prettyprint())

Problem name: Multi-State Knapsack Problem

Maximize
  3*state_0_0 + 5*state_0_1 + 2*state_0_2 + 7*state_1_0 + state_1_1
  + 6*state_1_2 + 8*state_2_0 + 2*state_2_1 + 4*state_2_2

Subject to
  Linear constraints (4)
    state_0_0 + state_1_0 + state_2_0 == 1  'state_assignment_constraint_0'
    state_0_1 + state_1_1 + state_2_1 == 1  'state_assignment_constraint_1'
    state_0_2 + state_1_2 + state_2_2 == 1  'state_assignment_constraint_2'
    2*state_0_0 + 3*state_0_1 + 5*state_0_2 + 4*state_1_0 + 2*state_1_1
    + state_1_2 + 3*state_2_0 + 4*state_2_1 + 2*state_2_2
    + total_cost_constraint@int_slack@0 + 2*total_cost_constraint@int_slack@1
    + 4*total_cost_constraint@int_slack@2 + 3*total_cost_constraint@int_slack@3
    == 10  'total_cost_constraint'

  Binary variables (13)
    state_0_0 state_1_0 state_2_0 state_0_1 state_1_1 state_2_1 state_0_2
    state_1_2 state_2_2 total_cost_constraint@int_slack@0
    total_cost_constraint@int_slack@1 total_cost_constraint@int_slack@2
    

### 🔍 **Core Logic of `LinearEqualityToPenalty`**  

**Goal:**  
Convert **equality constraints** $\sum_{j} a_j x_j = b$ into **penalty terms** added to the objective function.  

---

### 🔑 **1. Penalty Term Decomposition**  
The penalty term for an equality constraint is:  
$$
\text{penalty} \cdot \left( \sum_{j} a_j x_j - b \right)^2
$$  
Expanding it gives:  
$$
\text{penalty} \cdot \left( \sum_{j} a_j x_j \right)^2 
- 2 \cdot \text{penalty} \cdot b \cdot \sum_{j} a_j x_j 
+ \text{penalty} \cdot b^2
$$  
This is split into:  
- **Constant term**: $\text{penalty} \cdot b^2$  
- **Linear term**: $-2 \cdot \text{penalty} \cdot a_j \cdot b$ for each $x_j$  
- **Quadratic term**: $\text{penalty} \cdot a_j \cdot a_k$ for each $x_j x_k$  

---

### 🔑 **2. Key Functions**  

#### **1️⃣ `convert(problem: QuadraticProgram)`**  
1. **Penalty Setup**: If not provided, it is auto-calculated.  
2. **Variable Copy**: Copy problem variables to a new problem.  
3. **Constraint Conversion**:  
   - **Constant term**:  
     $$
     \text{offset} += \text{penalty} \cdot b^2
     $$
   - **Linear term**:  
     $$
     \text{linear}[j] += -2 \cdot \text{penalty} \cdot a_j \cdot b
     $$
   - **Quadratic term**:  
     $$
     \text{quadratic}[(j, k)] += \text{penalty} \cdot a_j \cdot a_k
     $$  

In [95]:
from qiskit_optimization.converters.linear_equality_to_penalty import LinearEqualityToPenalty

equality_penalty_problem = LinearEqualityToPenalty(penalty=100).convert(binary_problem)
print(equality_penalty_problem.prettyprint())

Problem name: Multi-State Knapsack Problem

Maximize
  -500*state_0_0^2 - 1200*state_0_0*state_0_1 - 2000*state_0_0*state_0_2
  - 1800*state_0_0*state_1_0 - 800*state_0_0*state_1_1 - 400*state_0_0*state_1_2
  - 1400*state_0_0*state_2_0 - 1600*state_0_0*state_2_1
  - 800*state_0_0*state_2_2 - 400*state_0_0*total_cost_constraint@int_slack@0
  - 800*state_0_0*total_cost_constraint@int_slack@1
  - 1600*state_0_0*total_cost_constraint@int_slack@2
  - 1200*state_0_0*total_cost_constraint@int_slack@3 - 1000*state_0_1^2
  - 3000*state_0_1*state_0_2 - 1400*state_0_1*state_1_1
  - 600*state_0_1*state_1_2 - 2600*state_0_1*state_2_1
  - 1200*state_0_1*state_2_2 - 600*state_0_1*total_cost_constraint@int_slack@0
  - 1200*state_0_1*total_cost_constraint@int_slack@1
  - 2400*state_0_1*total_cost_constraint@int_slack@2
  - 1800*state_0_1*total_cost_constraint@int_slack@3 - 2600*state_0_2^2
  - 1200*state_0_2*state_1_2 - 2200*state_0_2*state_2_2
  - 1000*state_0_2*total_cost_constraint@int_slack@0
  - 2

### 🧮 **Comparison of Key Terms Before and After Conversion**  

---

### **1️⃣ Linear Constraints (Before Conversion)**
The problem has the following constraints before conversion:  
1. $\text{state}_{0,0} + \text{state}_{1,0} + \text{state}_{2,0} = 1$  
2. $\text{state}_{0,1} + \text{state}_{1,1} + \text{state}_{2,1} = 1$  
3. $\text{state}_{0,2} + \text{state}_{1,2} + \text{state}_{2,2} = 1$  
4. $2 \cdot \text{state}_{0,0} + 3 \cdot \text{state}_{0,1} + 5 \cdot \text{state}_{0,2} + 4 \cdot \text{state}_{1,0} + 2 \cdot \text{state}_{1,1} + \text{state}_{1,2} + 3 \cdot \text{state}_{2,0} + 4 \cdot \text{state}_{2,1} + 2 \cdot \text{state}_{2,2} + $  
   $\text{total\_cost\_constraint@int\_slack@0} + 2 \cdot \text{total\_cost\_constraint@int\_slack@1} + 4 \cdot \text{total\_cost\_constraint@int\_slack@2} + 3 \cdot \text{total\_cost\_constraint@int\_slack@3} = 10$  

---

### **2️⃣ Constraint Representation (After Conversion)**
All the constraints are converted into **penalty terms** added to the objective function.  
For simplicity, we highlight 2 constraints.  
 
- **After**:  
  $$
  -500 \cdot \text{state}_{0,0}^2 - 1700 \cdot \text{state}_{1,0}^2 - 1000 \cdot \text{state}_{2,0}^2 - 2600 \cdot \text{state}_{1,0} \cdot \text{state}_{2,0} - 1800 \cdot \text{state}_{0,0} \cdot \text{state}_{1,0} - 1400 \cdot \text{state}_{0,0} \cdot \text{state}_{2,0}
  $$  

In [96]:
from qiskit_optimization.converters.flip_problem_sense import MaximizeToMinimize

minimize_problem = MaximizeToMinimize().convert(equality_penalty_problem)
print(minimize_problem.prettyprint())

Problem name: Multi-State Knapsack Problem

Minimize
  500*state_0_0^2 + 1200*state_0_0*state_0_1 + 2000*state_0_0*state_0_2
  + 1800*state_0_0*state_1_0 + 800*state_0_0*state_1_1 + 400*state_0_0*state_1_2
  + 1400*state_0_0*state_2_0 + 1600*state_0_0*state_2_1
  + 800*state_0_0*state_2_2 + 400*state_0_0*total_cost_constraint@int_slack@0
  + 800*state_0_0*total_cost_constraint@int_slack@1
  + 1600*state_0_0*total_cost_constraint@int_slack@2
  + 1200*state_0_0*total_cost_constraint@int_slack@3 + 1000*state_0_1^2
  + 3000*state_0_1*state_0_2 + 1400*state_0_1*state_1_1
  + 600*state_0_1*state_1_2 + 2600*state_0_1*state_2_1
  + 1200*state_0_1*state_2_2 + 600*state_0_1*total_cost_constraint@int_slack@0
  + 1200*state_0_1*total_cost_constraint@int_slack@1
  + 2400*state_0_1*total_cost_constraint@int_slack@2
  + 1800*state_0_1*total_cost_constraint@int_slack@3 + 2600*state_0_2^2
  + 1200*state_0_2*state_1_2 + 2200*state_0_2*state_2_2
  + 1000*state_0_2*total_cost_constraint@int_slack@0
  + 20

## 🔥 **Key Logic of `to_ising` Function**

### 🧪 **1. Input Validation**
- **Binary Variable Check**: Ensures all variables are binary. If not, it raises an error.  
- **Constraint Check**: Ensures the problem has no constraints. If constraints exist, it raises an error.  

---

### 🧪 **2. Objective Function Conversion**
#### 🔹 **Constant Term**  
- Directly added to the **offset**.  

#### 🔹 **Linear Terms** ($c_i x_i$)  
- Each binary variable $x_i$ is converted to a Pauli-Z term:  
  $$
  c_i x_i = \frac{c_i}{2} (1 - z_i)
  $$
  - $-\frac{c_i}{2} z_i$ is added to the qubit operator.  
  - $\frac{c_i}{2}$ is added to the offset.  

#### 🔹 **Quadratic Terms** ($c_{ij} x_i x_j$)  
- Each product $x_i x_j$ is converted to a Pauli-Z interaction:  
  $$
  c_{ij} x_i x_j = \frac{c_{ij}}{4} (1 - z_i - z_j + z_i z_j)
  $$
  - $\frac{c_{ij}}{4}$ is added to the **offset**.  
  - $-\frac{c_{ij}}{4} z_i$ and $-\frac{c_{ij}}{4} z_j$ are added as single Pauli-Z operators.  
  - $\frac{c_{ij}}{4} z_i z_j$ is added as a two-qubit Pauli-Z interaction.  



## 🔥 **Summary of Key Concepts**
| **Step**           | **Logic**                                           |
|-------------------|-----------------------------------------------------|
| **Input Validation** | Check for non-binary variables and constraints.     |
| **Initialization**  | Initialize lists for operators, offsets, etc.       |
| **Constant Term**    | Add the constant part of the objective to the offset. |
| **Linear Terms**     | Convert $\mathbf{c}^T \mathbf{x}$ to Ising using $z$. |
| **Quadratic Terms**  | Convert $\mathbf{x}^T Q \mathbf{x}$ to Pauli-Z operators. |



##### How can we solve QUBOs on quantum computers?

- Quantum computers require for us to formulate our problems as **Hamiltonian problems**.

- Hamiltonian: mathematical description of a physical system's **energy**, in terms of operators or matrices. For an eigenstate $|x\rangle$ of the system,

$$H|x\rangle = E_{|x\rangle}|x\rangle \quad E_{|x\rangle} = \langle x|H|x\rangle$$

- Goal: to find a Hamiltonian, $H_C$, which **encodes our cost function**, $C(x)$.

$$H_C|x\rangle = C(x)|x\rangle$$

- If we can encode $C(x)$ in some $H_C$, we can then minimize it by finding the system's lowest energy state, i.e. the system's **ground state**

$$|x_{\text{optimal}}\rangle = \underset{|x\rangle \in \mathcal{H}}{\text{argmin }} E_{|x\rangle}$$

##### Mapping to a Hamiltonian

So, how do we find $H_C$?

1. Map each of the optimization variables to a qubit using the substitution:
   
   $$x_i = \frac{1-z_i}{2}, \quad z_i \in \{-1,1\}$$

2. Promote $z_i$ to a Pauli spin operator $Z_i$, where $Z_i|x_i\rangle = (-1)^{x_i} |x_i\rangle$. This is equivalent to the above since $(-1)^{x_i} = 1 - 2x_i$ for $x_i \in \{0,1\}$.

3. Make the substitution:

$$\begin{aligned}
H_C &= \sum_{ij} Q_{ij} \frac{1-Z_i}{2} \frac{1-Z_j}{2} + \sum_i \mu_i \frac{1-Z_i}{2} \\
&= \sum_{ij} \frac{Q_{ij}}{4} - \sum_i \left(\sum_j \frac{Q_{ij}}{4}\right)Z_i - \sum_j \left(\sum_i \frac{Q_{ij}}{4}\right)Z_j + \sum_{ij} \frac{Q_{ij}}{4} Z_iZ_j + \sum_i \frac{\mu_i}{2} - \sum_i \frac{\mu_i}{2} Z_i \\
&= \sum_{ij} \frac{Q_{ij}}{4} Z_iZ_j - \sum_i \frac{1}{2}\left(\mu_i + \sum_j Q_{ij}\right)Z_i + \sum_{ij} \frac{Q_{ij}}{4} + \sum_i \frac{\mu_i}{2}
\end{aligned}$$

Where $Q_{ij}$ is symmetric, $i \leftrightarrow j$


In [97]:
qaoa = QAOA(sampler=Sampler(), optimizer=COBYLA())
optimizer = MinimumEigenOptimizer(qaoa)

operator, offset = minimize_problem.to_ising()

C:\Users\forca\AppData\Local\Temp\ipykernel_23196\618527468.py:1: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  qaoa = QAOA(sampler=Sampler(), optimizer=COBYLA())


In [98]:
offset

np.float64(9631.0)

In [99]:
operator

SparsePauliOp(['IIIIIIIIIIIIZ', 'IIIIIIIIIIIZI', 'IIIIIIIIIIZII', 'IIIIIIIIIZIII', 'IIIIIIIIZIIII', 'IIIIIIIZIIIII', 'IIIIIIZIIIIII', 'IIIIIZIIIIIII', 'IIIIZIIIIIIII', 'IIIZIIIIIIIII', 'IIZIIIIIIIIII', 'IZIIIIIIIIIII', 'ZIIIIIIIIIIII', 'IIIIIIIIIIIZZ', 'IIIIIIIIIIZIZ', 'IIIIIIIIIZIIZ', 'IIIIIIIIZIIIZ', 'IIIIIIIZIIIIZ', 'IIIIIIZIIIIIZ', 'IIIIIZIIIIIIZ', 'IIIIZIIIIIIIZ', 'IIIZIIIIIIIIZ', 'IIZIIIIIIIIIZ', 'IZIIIIIIIIIIZ', 'ZIIIIIIIIIIIZ', 'IIIIIIIIIIZZI', 'IIIIIIIIIZIZI', 'IIIIIIIIZIIZI', 'IIIIIIIZIIIZI', 'IIIIIIZIIIIZI', 'IIIIIZIIIIIZI', 'IIIIZIIIIIIZI', 'IIIZIIIIIIIZI', 'IIZIIIIIIIIZI', 'IZIIIIIIIIIZI', 'ZIIIIIIIIIIZI', 'IIIIIIIIIZZII', 'IIIIIIIIZIZII', 'IIIIIIIZIIZII', 'IIIIIIZIIIZII', 'IIIIIZIIIIZII', 'IIIIZIIIIIZII', 'IIIZIIIIIIZII', 'IIZIIIIIIIZII', 'IZIIIIIIIIZII', 'ZIIIIIIIIIZII', 'IIIIIIIIZZIII', 'IIIIIIIZIZIII', 'IIIIIIZIIZIII', 'IIIIIZIIIZIII', 'IIIIZIIIIZIII', 'IIIZIIIIIZIII', 'IIZIIIIIIZIII', 'IZIIIIIIIZIII', 'ZIIIIIIIIZIII', 'IIIIIIIZZIIII', 'IIIIIIZIZIIII', 'IIIIIZIIZIIII',

In [102]:
profits = [[3, 5, 2],
           [7, 1, 6],
           [8, 2, 4]]
costs = [[2, 3, 5],
         [4, 2, 1],
         [3, 4, 2]]


max_total_cost = 10


solution, max_profit = solve_multi_state_knapsack(profits, costs, max_total_cost)
print(f"Optimal state selection: {solution}, Maximum profit: {max_profit}")


Optimal state selection: [2, 0, 1], Maximum profit: 19


In [ ]:
original_problem = formulate_qp_multi_state_knapsack(profits, costs, max_total_cost)

# Create instances of the conversion classes
linear_to_penalty_converter = LinearInequalityToPenalty()
inequality_to_equality_converter = InequalityToEquality(mode="integer")
integer_to_binary_converter = IntegerToBinary()
linear_equality_to_penalty_converter = LinearEqualityToPenalty()
maximize_to_minimize_converter = MaximizeToMinimize()

# Store the converters in a list
_converters = [
    linear_to_penalty_converter,
    inequality_to_equality_converter,
    integer_to_binary_converter,
    linear_equality_to_penalty_converter,
    maximize_to_minimize_converter
]

# Perform the conversions step by step
binary_penalty_problem = linear_to_penalty_converter.convert(original_problem)
equality_problem = inequality_to_equality_converter.convert(binary_penalty_problem)
binary_problem = integer_to_binary_converter.convert(equality_problem)
equality_penalty_problem = linear_equality_to_penalty_converter.convert(binary_problem)
minimize_problem = maximize_to_minimize_converter.convert(equality_penalty_problem)

qaoa = QAOA(sampler=Sampler(), optimizer=COBYLA())
optimizer = MinimumEigenOptimizer(qaoa)

operator, offset = minimize_problem.to_ising()

eigen_result = optimizer._min_eigen_solver.compute_minimum_eigenvalue(operator)

raw_samples = optimizer._eigenvector_to_solutions(
    eigen_result.eigenstate, minimize_problem
)
raw_samples.sort(key=lambda x: x.fval)


samples, best_raw = optimizer._interpret_samples(original_problem, raw_samples, _converters)

solution = best_raw.x

# Extract results
total_profit = 0
assignments = []

n = len(profits[0])  # Number of items
num_states = len(profits)  # Number of possible states per item


for i in range(n):
    for s in range(num_states):
        if solution[num_states * i + s] == 1:
            total_profit += profits[s][i]
            assignments.append(s)

print(f"Optimal state selection: {assignments}, Maximum profit: {total_profit}")

C:\Users\forca\AppData\Local\Temp\ipykernel_23196\2984190239.py:30: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  qaoa = QAOA(sampler=Sampler(), optimizer=COBYLA())


Optimal state selection: [2, 0, 1], Maximum profit: 19
